In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./gap"))

# Any results you write to the current directory are saved as output.

['constants.py', 'CONTRIBUTING.md', 'gap-development.tsv', 'gap-test.tsv', 'gap-validation.tsv', 'gap_scorer.py', 'LICENSE', 'README.md', 'sample_submission_stage_1.csv', 'test_stage_1.tsv', 'test_stage_1.tsv.zip']


In [2]:
#Getting train data from github google repo

gh_test = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv", delimiter='\t')
gh_valid = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv", delimiter='\t')
train = pd.concat((gh_test, gh_valid)).rename(columns={'A': 'A_Noun', 'B': 'B_Noun'}).reset_index(drop=True)
train.shape

(2454, 11)

In [ ]:
test_df = pd.read_csv("./gap/test_stage_1.tsv", delimiter='\t').rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})
submission = pd.read_csv('./gap/sample_submission_stage_1.csv')

In [ ]:
#replace function
def name_replace(s, r1, r2):
    s = str(s).replace(r1,r2)
#    for r3 in r1.split(' '):
#        s = str(s).replace(r3,r2)
    return s

def dot_exist(s, poff, noff):
    dot_list = [m.start() for m in re.finditer('\.', s)]

In [ ]:
import string
def get_features(df):
    df['char_count'] = df['Text'].apply(len)
    df['word_count'] = df['Text'].apply(lambda x: len(x.split()))
    df['word_density'] = df['char_count'] / (df['word_count']+1)
    df['punctuation_count'] = df['Text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
    df['title_word_count'] = df['Text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
    df['upper_case_word_count'] = df['Text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
    df['section_min'] = df[['Pronoun-offset', 'A-offset', 'B-offset']].min(axis=1)
    df['Pronoun-offset2'] = df['Pronoun-offset'] + df['Pronoun'].map(len)
    df['A-offset2'] = df['A-offset'] + df['A_Noun'].map(len)
    df['B-offset2'] = df['B-offset'] + df['B_Noun'].map(len)                               
    df['section_max'] = df[['Pronoun-offset2', 'A-offset2', 'B-offset2']].max(axis=1)
    #df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['A_Noun'], 'anoun'), axis=1)
    #df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['B_Noun'], 'bnoun'), axis=1)
    #df['A_count'] = df.Text.str.count(df['A_Noun'])
    #df['B_count'] = df.Text.str.count(df['B_Noun'])
    df['A-dist'] = (df['Pronoun-offset'] - df['A-offset']).abs()
    df['B-dist'] = (df['Pronoun-offset'] - df['B-offset']).abs()
    return(df)

train = get_features(train)
test = get_features(test_df)

In [100]:

def coref(row):
    raw_text = row['clear'] 
    doc = nlp(raw_text)
    return str(doc._.coref_clusters)

def corefa(row):
    raw_text = row['clear'] 
    doc = nlp(raw_text[row['A-offset']:])
    return str(doc._.coref_clusters)

def corefb(row):
    raw_text = row['clear'] 
    doc = nlp(raw_text[row['B-offset']:])
    return str(doc._.coref_clusters)


# Apply a first round of text cleaning techniques
import re
import string

def clear(row):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = row['Text']
    #text = text.lower()
    text = re.sub('\(.*?\)', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

#round1 = lambda x: clean_text_round1(x)

In [96]:
datatest = train.head()
datatest

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,A-coref,B_Noun,B-offset,B-coref,URL
0,test-1,"Upon their acceptance into the Kontinental Hockey League, Dehner left Finland to sign a contract in Germany with EHC M*nchen of the DEL on June 18...",His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four local titles, and became one of the most expensive teams in the world, with a first team (Alonso- Luqu...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,"Though his emigration from the country has affected his leadership status, Kamel is still a respected elder of the clan. After the fall of Hussien...",He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have made promises to us are called Bernardo Mattarella, Prince Alliata, the monarchist MP Marchesano an...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore patrollers (SPs) (Tom Berenger and William McNamara) who must escort a beautiful prisoner (Erika El...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [101]:
# Let's take a look at the updated text
#data_clean = pd.DataFrame(datatest.apply(round1))
#data_clean

# Use .apply to save the new column if we'd like
#datatest = pd.DataFrame(test.Text.apply(round1))
datatest['clear'] = datatest.apply(clear, axis=1)
datatest['coref'] = datatest.apply(coref, axis=1)
datatest['corefa'] = datatest.apply(corefa, axis=1)
datatest['corefb'] = datatest.apply(corefb, axis=1)
datatest[['coref', 'corefa', 'corefb']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

,coref,corefa,corefb
0,"[he: [he, His]]","[Bob Suter: [Bob Suter, His]]","[Dehner's uncles: [Dehner's uncles, His]]"
1,None,None,None
2,"[Kamel: [his, his, Kamel], the clan: [the clan, the clan], Saddam: [Saddam, He, his, his]]","[Ali Aladhadh: [Ali Aladhadh, He, his, his]]","[Saddam: [Saddam, He, his, his]]"
3,"[the massacre of Portella di Ginestra: [the massacre of Portella di Ginestra, the massacre], Giuliano: [Giuliano, Giuliano, Giuliano], a trial whi...","[Alliata and Marchesano: [Alliata and Marchesano, their], a trial which dealt with their alleged role in the event: [a trial which dealt with thei...","[Pisciotta: [Pisciotta, he], Giuliano: [Giuliano, Giuliano]]"
4,"[Eddie Devane : [Eddie Devane , his, his, his]]",None,None


In [80]:
pd.options.display.max_colwidth =150

In [ ]:
# Loading NLP libraries
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import nltk

In [ ]:
def get_nlp_features(s, w):
    doc = nlp(str(s))
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    return len(tokens[((tokens['text']==w) & (tokens['dep']=='poss'))])

train['A-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'anoun'))
train['B-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'bnoun'))
test['A-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'anoun'))
test['B-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'bnoun'))

In [44]:
train.head()

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,A-coref,B_Noun,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [102]:
train['Text'][0]

"Upon their acceptance into the Kontinental Hockey League, Dehner left Finland to sign a contract in Germany with EHC M*nchen of the DEL on June 18, 2014. After capturing the German championship with the M*nchen team in 2016, he left the club and was picked up by fellow DEL side EHC Wolfsburg in July 2016. Former NHLer Gary Suter and Olympic-medalist Bob Suter are Dehner's uncles. His cousin is Minnesota Wild's alternate captain Ryan Suter."

In [3]:
#Adding nueral coref
import spacy
nlp = spacy.load('en_coref_md')

#doc = nlp(str(train['Text'][0]))

doc._.has_coref
doc._.coref_clusters

[he: [he, His]]

In [84]:
doc = nlp(u"Though his emigration from the country has affected his leadership status, Kamel is still a respected elder of the clan. After the fall of Hussien's regime, many considered Dr. Ali Aladhadh a candidate to lead the clan. A contributor to Iraq's liberation, Ali Aladhadh and a long time oppose to Saddam's regime. He was ambushed with his pregnant wife on his way to the hospital in 2006 by Iraqi insurgents.")

#doc._.has_coref
doc._.coref_clusters

[Kamel: [his, his, Kamel],
 the clan: [the clan, the clan],
 Saddam: [Saddam, He, his, his]]

In [88]:
word = "Though his emigration from the country has affected his leadership status, Kamel is still a respected elder of the clan. After the fall of Hussien's regime, many considered Dr. Ali Aladhadh a candidate to lead the clan. A contributor to Iraq's liberation, Ali Aladhadh and a long time oppose to Saddam's regime. He was ambushed with his pregnant wife on his way to the hospital in 2006 by Iraqi insurgents."
#word.find("Dehner")
#word[word.find("Dehner"):]

doc = nlp(word[295:])

doc._.has_coref
doc._.coref_clusters

[Saddam: [Saddam, He, his, his]]

In [103]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(word)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Though though ADP IN mark Xxxxx True False
his -PRON- ADJ PRP$ poss xxx True True
emigration emigration NOUN NN nsubj xxxx True False
from from ADP IN prep xxxx True True
the the DET DT det xxx True True
country country NOUN NN pobj xxxx True False
has have VERB VBZ aux xxx True True
affected affect VERB VBN advcl xxxx True False
his -PRON- ADJ PRP$ poss xxx True True
leadership leadership NOUN NN compound xxxx True False
status status NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
Kamel kamel PROPN NNP nsubj Xxxxx True False
is be VERB VBZ ROOT xx True True
still still ADV RB advmod xxxx True True
a a DET DT det x True True
respected respected ADJ JJ amod xxxx True False
elder elder NOUN NN attr xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
clan clan NOUN NN pobj xxxx True False
. . PUNCT . punct . False False
After after ADP IN prep Xxxxx True False
the the DET DT det xxx True True
fall fall NOUN NN pobj xxxx True False
of of ADP IN pre

In [ ]:
def get_features3(df):
    doc = nlp(word)
    doc._.has_coref
    doc._.coref_clusters
    
    doc = nlp(str(s))
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    return len(tokens[((tokens['text']==w) & (tokens['dep']=='poss'))])
    return(df)

op = get_features3(train)

In [ ]:
df['cluster'] = train['Text'].apply(lambda x: [wrd for wrd in x.split() if wrd.isupper()])

In [ ]:
train = train.rename(columns={'A-coref':'A', 'B-coref':'B'})
train['A'] = train['A'].astype(int)
train['B'] = train['B'].astype(int)
train['NEITHER'] = 1.0 - (train['A'] + train['B'])

In [ ]:
train['B_Noun'][0]

In [ ]:


def get_features2(df):
    doc = nlp(str(s))
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    return len(tokens[((tokens['text']==w) & (tokens['dep']=='poss'))])
    return(df)

train = get_features2(train)
test = get_features2(test)

In [ ]:
def get_features3(trainDF):

    return(trainDF)

train = get_features3(train)
test = get_features3(test)

In [ ]:
train.head()

In [ ]:
from sklearn import *
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

col = ['char_count','word_count','word_density','punctuation_count','title_word_count','upper_case_word_count','section_min', 'Pronoun-offset2', 'A_count', 'B_count', 'A-offset2', 'B-offset2', 'section_max', 'A-poss', 'B-poss', 'A-dist', 'B-dist']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)
x1.head()

In [ ]:
model = multiclass.OneVsRestClassifier(ensemble.RandomForestClassifier(max_depth = 7, n_estimators=1000, random_state=33))
# model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

# param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
# model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

In [ ]:
def my_model(x1, x2, y1, y2):
    model.fit(x1, y1)
    log_loss = metrics.log_loss(y2, model.predict_proba(x2))
    print('log_loss', log_loss)
    return log_loss

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
# Build step forward feature selection
sfs1 = sfs(model,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(x1, y1)

In [ ]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

In [ ]:
print(col)

In [ ]:
col = ['B_count', 'A-poss', 'B-poss']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

In [ ]:
col = ['B_count', 'A-poss', 'B-poss']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)

#model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])

results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

In [ ]:
import pickle
# Let's pickle it for later use
train.to_pickle("train.pkl")
test.to_pickle("test.pkl")